## Eager Executaion

In [1]:
import tensorflow as tf

tf.enable_eager_execution()

In [4]:
print(tf.add(1,2))
print(tf.add([1,2],[3,4]))
print(tf.square(5))
print(tf.reduce_sum([1,2,3]))
print(tf.encode_base64("hello world"))

print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


In [5]:
x = tf.matmul([[1]], [[2,3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


In [7]:
import numpy as np
ndarray = np.ones([3,3])

tensor = tf.multiply(ndarray, 42)
print(tensor)
print(np.add(tensor,1))
print(tensor.numpy())

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [8]:
x = tf.random_uniform([3,3])
print(tf.test.is_gpu_available())
print(x.device.endswith('GPU:0'))

False
False


In [9]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x,x)
    result = time.time() - start
    print("10 loops: {:0.2f}ms".format(1000*result))

In [10]:
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random_uniform([1000,1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

if tf.test.is_gpu_available():
    with tf.device("GPU:0"):
        x = tf.random_uniform([1000,1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
10 loops: 112.12ms


In [11]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

In [12]:
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line1
    Line2
    Line3
    """)

ds_file = tf.data.TextLineDataset(filename)

In [15]:
print(filename)

/var/folders/r4/ld0lj9qx2r9dbdjl8zmcyzqm0000gn/T/tmp931qs_6a


In [17]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)
ds_file = ds_file.batch(2)

In [20]:
for x in ds_tensors:
    print(x)

for x in ds_file:
    print(x)

tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 16], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)
tf.Tensor([b'Line1' b'    Line2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line3' b'    '], shape=(2,), dtype=string)


## Automatic Differentiation

In [21]:
import tensorflow as tf

tf.enable_eager_execution()

In [22]:
x = tf.ones((2,2))
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)

dz_dx = t.gradient(z,x)
for i in [0,1]:
    for j in [0,1]:
        assert dz_dx[i][j].numpy() == 8.0

In [23]:
x = tf.ones((2,2))
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)

dz_dx = t.gradient(z,y)
assert dz_dx.numpy() == 8.0

In [28]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x*x
    z = y*y
dz_dx = t.gradient(z,x)
dy_dx = t.gradient(y,x)
del t

In [29]:
print(dz_dx)

tf.Tensor(108.0, shape=(), dtype=float32)


In [39]:
def f(x,y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output, x)
    return output

In [31]:
def grad(x,y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x,y)
    return t.gradient(out, x)

In [41]:
x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0 # 3x^2 with x=2
assert grad(x, 5).numpy() == 12.0 # 3x^2 with x=2
assert grad(x, 4).numpy() == 4.0 # 2x^1 with x=2

In [42]:
x = tf.Variable(1.0)

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
    dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0 # 3x^2 with x=1
assert d2y_dx2.numpy() == 6.0 # 2 * (3x^1) with x=1

## Custom training